# Student Loan Risk with Deep Learning

In [2]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path
tf.compat.v1.losses.sparse_softmax_cross_entropy


<function tensorflow.python.ops.losses.losses_impl.sparse_softmax_cross_entropy(labels, logits, weights=1.0, scope=None, loss_collection='losses', reduction='weighted_sum_by_nonzero_weights')>

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [3]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [4]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [5]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y[:5]

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [7]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.copy()
X = X.drop("credit_ranking", axis=1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [8]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [9]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [16]:
# Define the the number of inputs (features) to the model
input_features = len(X.columns)

# Review the number of features
input_features

11

In [19]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=6, activation="relu", input_dim=input_features))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=3, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [20]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 6)                 72        
                                                                 
 dense_7 (Dense)             (None, 3)                 21        
                                                                 
 dense_8 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [22]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
tf.compat.v1.train.Optimizer

tensorflow.python.training.optimizer.Optimizer

In [24]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)
tf.compat.v1.ragged.RaggedTensorValue

Epoch 1/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5086 - accuracy: 0.7573
Epoch 2/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5085 - accuracy: 0.7590
Epoch 3/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5081 - accuracy: 0.7573
Epoch 4/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5077 - accuracy: 0.7581
Epoch 5/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5075 - accuracy: 0.7565
Epoch 6/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5073 - accuracy: 0.7590
Epoch 7/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5073 - accuracy: 0.7573
Epoch 8/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5067 - accuracy: 0.7581
Epoch 9/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5067 - accuracy: 0.7581
Epoch 10/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5064 - accuracy: 0.7590
Epoch 11/50
38/38 [

tensorflow.python.ops.ragged.ragged_tensor_value.RaggedTensorValue

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [25]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5413 - accuracy: 0.7350 - 205ms/epoch - 16ms/step
Loss: 0.5412517786026001, Accuracy: 0.7350000143051147


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [26]:
# Set the model's file path
file_path = Path("student_loan_model")

# Export your model to a keras file
nn_model.save(file_path)

INFO:tensorflow:Assets written to: student_loan_model\assets


INFO:tensorflow:Assets written to: student_loan_model\assets


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [28]:
# Set the model's file path
file_path = Path("student_loan_model")

# Load the model to a new object
new_model = tf.keras.models.load_model(file_path)
tf.io.gfile.exists

<function tensorflow.python.lib.io.file_io.file_exists_v2(path)>

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [29]:
# Make predictions with the test data
predictions = nn_model.predict(X_test_scaled)

# Display a sample of the predictions
predictions[:5]

13/13 [==============================] - 0s 1ms/step


array([[0.57575464],
       [0.2682382 ],
       [0.7595768 ],
       [0.6984814 ],
       [0.98604023]], dtype=float32)

In [33]:
import numpy as np
import pandas as pd

In [34]:
# Save the predictions to a DataFrame and round the predictions to binary results
rounded_predictions = np.round(predictions).astype(int)
predictions_df = pd.DataFrame(rounded_predictions, columns=["predictions"])
predictions_df

,predictions
0,1
1,0
2,1
3,1
4,1
...,...
395,1
396,0
397,1
398,0


### Step 4: Display a classification report with the y test data and predictions

In [35]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df))

              precision    recall  f1-score   support

           0       0.71      0.74      0.72       188
           1       0.76      0.73      0.75       212

    accuracy                           0.73       400
   macro avg       0.73      0.74      0.73       400
weighted avg       0.74      0.73      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**



In [ ]:
# Question 1: 
# The data that should be collected to build a recommendation system for student loan options should include the following:
# - Financial information such as income, expenses, and credit score
# - Academic information such as GPA, major, and school
# - Preferences and goals such as understanding a student's career goals, desired loan terms and risk tolerance. 
# This data should be collected because it can help determine the student's financial situation to help assess the financial capacity of the student and family to repay the loans. 
# Academic performance may indicate the student's potential fo success and eligility for loans and scholarships. 
# And preferences and goals can help tailor the loan options to the student's needs and goals.
# The type of data described is appropriate for a recommendation system for student loan options because it can help match students with the best loan options 
# because by collecting this data a tailored recommendation can be made based on each individual's unique circumstances and goals. Such as the student's and family's ability 
# to repay loans, match astudent with the loan option that best fits their needs and ensures compliance with regulations governing loans and financial services
# to safeguard privacy and legal requirements. 

In [ ]:
# Question 2:
# Collaborative filtering is the recommended approach since it analyzes the interactions and preferences of multiple users to make appropriate suggestions for a specific user
# Collaborative filtering is the choice because it can personalize recommendations based on the behavior of similar users and can be used to recommend student loan options based on the preferences and behavior of other students with similar financial and academic profiles.
# Also, with scalability and flexibility, this type of filtering can handle large datasets and adapt to changing user preferences and behaviors over time.
# Lastly, feedback on loan options may not be available thus, collaborative filtering can generate suggestions based on the behavior of similar users without requiring explicit feedback on loan options.
# This approach is suitable for a recommendation system for student loan options because it can provide personalized recommendations based on the behavior of similar users and adapt to changing preferences and behaviors over time.

In [ ]:
# Question 3: 
# The first real world challenge is data privacy. Student loan recommendation systems rely on collecting and analyzing sensitive personal and financial data 
# from students, such as income, assets, academic performance, and demographic details. Given the sensitivity of this data, ensuring privacy and data security is paramount. 
# Any breach of privacy could lead to severe consequences for students, including identity theft, fraud, or unauthorized use of personal information.
# The second real world challenge is bias in recomemndations. Recommendation systems can inadvertently perpetuate biases present in the data used to train them. 
# In the context of student loans, bias can arise from various sources, including historical lending practices, socioeconomic factors, and institutional policies. 
# Biased recommendations can disproportionately target certain group of students, such as those from underrepresented communities or low-income backgrounds, 
# by limiting their access to favorable loan options or perpetuating discriminatory practices.